In [1]:
import pandas as pd
import numpy as np
import docplex
from docplex.mp.model import Model
from docplex.mp.progress import *
import time
import sys
import math
from sklearn.model_selection import train_test_split
import math

In [2]:
def DataPreperation(datasetname,sampleAmount, featCART, seed):
    global numericalGroup
    numericalGroup = []
    global data
    global trainData
    global samplePos
    global trainDataDummies
    global groupsOfFeatures
    global featureToGroup
    global testData
    global testDataDummies
    global samplePosTest
    # Read in prefered dataset
    if(datasetname == 'chess'):
        dataChess = pd.read_csv("kr-vs-kp.data", header=0, names=['bkblk','bknwy','bkon8','bkona','bkspr','bkxbq','bkxcr','bkxwp','blxwp','bxqsq','cntxt','dsopp','dwipd','hdchk','katri','mulch','qxmsq','r2ar8','reskd','reskr','rimmx','rkxwp','rxmsq','simpl','skach','skewr','skrxp','spcop','stlmt','thrsk','wkcti','wkna8','wknck','wkovl','wkpos','wtoeg','white_win'])
        
        if featCART:
            dataChess = dataChess[['white_win','rimmx','wknck','bxqsq','wkna8', 'wkpos','bkxbq','katri', 'bkblk']]
        data, testData = train_test_split(dataChess.dropna(), train_size=sampleAmount, random_state=seed)
        
        trainData = data.drop(columns=['white_win'])
        samplePos = (data['white_win'] == 'won')*1
        
        samplePosTest = (testData['white_win'] == 'won')*1
        testData = testData.drop(columns='white_win')
    elif(datasetname == 'adult'):
        dataAdult = pd.read_csv("a1a.txt", delim_whitespace=True, header=None, names = ['salery50k','age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country']) 
        if featCART:
            dataAdult = dataAdult[['salery50k','relationship', 'occupation', 'education', 'captial-gain','workclass']]
        data, testData = train_test_split(dataAdult.dropna(), train_size=sampleAmount, random_state=seed)
        trainData = data.drop(columns='salery50k')
        samplePos = (data['salery50k'] == 1)*1
        
        samplePosTest = (testData['salery50k'] == 1)*1
        testData = testData.drop(columns='salery50k')
        
        numericalGroup = [1, 3, 5, 11, 12, 13]
    elif(datasetname == 'breast'):
        dataBreast = pd.read_csv("breast.txt", delim_whitespace=True, header=0, names=['Class','Sample code number','Clump Thickness','Uniformity of Cell Size','Uniformity of Cell Shape','Marginal Adhesion','Single Epithelial Cell Size','Bare Nuclei','Bland Chromatin','Normal Nucleoli','Mitoses'])
        dataBreast = dataBreast.drop(columns='Sample code number') #drop id-numbers of observations
        if featCART:
            dataBreast = dataBreast[['Class','Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Bare Nuclei']]
        data, testData = train_test_split(dataBreast.dropna(), train_size=sampleAmount, random_state=seed)
        
        #for i in trainData.columns: #take away the rownumber: of all variables.
        #        trainData[i] = trainData[i].str[trainData.iloc[0][i].index(':')+1:]
        samplePos = (data['Class'] == 2)*1
        trainData = data.drop(columns='Class')
        
        samplePosTest = (testData['Class'] == 2)*1
        testData = testData.drop(columns='Class')
        
        numericalGroup = [1, 2, 3, 4, 5, 6, 7, 8, 9]
    elif(datasetname == 'mushroom'):
        dataMushroom = pd.read_csv("agaricus-lepiota.data", header=0, names=['edible','cap-shape','cap-surface','cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])
        dataMushroom = dataMushroom.replace("?", np.nan)
        data = dataMushroom.dropna().sample(n=sampleAmount, random_state=seed)
        
        #data = dataMushroom.dropna().sample(n=5, random_state=1).drop(columns=['cap-color','bruises?','odor','gill-attachment','gill-spacing','gill-size','gill-color','stalk-shape','stalk-root','stalk-surface-above-ring','stalk-surface-below-ring','stalk-color-above-ring','stalk-color-below-ring','veil-type','veil-color','ring-number','ring-type','spore-print-color','population','habitat'])

        trainData = data.drop(columns='edible')
        samplePos = (data['edible'] == 'e')*1
    elif(datasetname == 'tictactoe'):
        dataTicTac = pd.read_csv("tic-tac-toe.data", header=None)
        data = dataTicTac.dropna().sample(n=sampleAmount, random_state=seed)
        trainData = data.drop(columns=9)
        samplePos = (data[9] == 'positive')*1
    elif(datasetname == 'monks'):
        dataMonks = pd.read_csv("monks-1.test", delim_whitespace=True, header = None)
        data = dataMonks.sample(n=sampleAmount, random_state=seed)
        for i in data.columns:
            data[i] = data[i].apply(str)
        trainData = data.drop(columns=[0, 7])
        samplePos = (data[0] == '1')*1
    elif(datasetname == 'votes'):
        dataVotes = pd.read_csv("house-votes-84.data", header=None)
        dataVotes = dataVotes.replace("?", np.nan)
        data = dataVotes.dropna().sample(n=sampleAmount, random_state=seed)
        trainData = data.drop(columns=0)
        samplePos = (data[0] == 'democrat')*1
    elif(datasetname == 'heart'):
        dataHeart1 = pd.read_csv("SPECT.test", header=None)
        dataHeart2 = pd.read_csv("SPECT.train", header=None)
        frames = [dataHeart1, dataHeart2]
        dataHeart = pd.concat(frames)
        data = dataHeart.dropna().sample(n=sampleAmount, random_state=seed)
        for i in data.columns:
            data[i] = data[i].apply(str)
        trainData = data.drop(columns=0)
        samplePos = (data[0] == '1')*1
    elif(datasetname == 'student'):
        studentMat = pd.read_csv("student-mat.csv", sep=";")
        studentMat['nonAlcoholic'] = np.where((studentMat['Dalc'] > 1) & (studentMat['Walc'] > 1), 0, 1)
        dataStudent = studentMat.drop(columns=['Walc', 'Dalc'])
        if (sampleAmount > 395):
            sampleAmount = math.floor(395*0.9)
        data, testData = train_test_split(dataStudent.dropna(), train_size=sampleAmount, random_state=seed)
        
        trainData = data.drop(columns='nonAlcoholic')
        samplePos = (data['nonAlcoholic'] == 1)*1    
        
        samplePosTest = (testData['nonAlcoholic'] == 1)*1
        testData = testData.drop(columns='nonAlcoholic')
        
        numericalGroup = [3, 7, 8, 13, 14, 15, 24, 25, 26, 27, 28]

    elif(datasetname == 'heloc'):
        dataHeloc = pd.read_csv("heloc_dataset_v1.csv", sep=",")        
        for i in dataHeloc.columns:
            if i != 'RiskPerformance':
                dataHeloc[i] = pd.qcut(dataHeloc[i], q=10, duplicates='drop')

        data, testData = train_test_split(dataHeloc.dropna(), train_size=sampleAmount, random_state=seed)
        trainData = data.drop(columns='RiskPerformance')
        samplePos = (data['RiskPerformance'] == 'Good')*1
        samplePosTest = (testData['RiskPerformance'] == 'Good')*1
        testData = testData.drop(columns='RiskPerformance')
        
        numericalGroup = [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23]
    else:
        print('ERROR: dataset name not found')
    #process data for tree

    start = 0
    for i in trainData.columns:
        if (start == 0):
            trainDataDummiesTemp = pd.get_dummies(trainData[i], prefix=i)
            trainDataDummies = trainDataDummiesTemp[sorted(trainDataDummiesTemp.columns.tolist(), key=str.lower)]
            start = 1
        else:
            trainDataDummiesTemp = pd.get_dummies(trainData[i], prefix=i)
            trainDataDummies = trainDataDummies.join(trainDataDummiesTemp[sorted(trainDataDummiesTemp.columns.tolist(), key=str.lower)])
    trainDataDummies = trainDataDummies.loc[:, (trainDataDummies != 0).any(axis=0)]
    
    start = 0
    for i in testData.columns:
        if (start == 0):
            testDataDummiesTemp = pd.get_dummies(testData[i], prefix=i)
            testDataDummies = testDataDummiesTemp[sorted(testDataDummiesTemp.columns.tolist(), key=str.lower)]
            start = 1
        else:
            testDataDummiesTemp = pd.get_dummies(testData[i], prefix=i)
            testDataDummies = testDataDummies.join(testDataDummiesTemp[sorted(testDataDummiesTemp.columns.tolist(), key=str.lower)])
    testDataDummies = testDataDummies.loc[:, (testDataDummies != 0).any(axis=0)]
    
    groupsN = trainData.shape[1]
    G = range(1, groupsN + 1)
    groupsOfFeatures = trainData.nunique()
    featureToGroup = []
    for g in G:
        amountOfFeatures = groupsOfFeatures.iloc[g-1]
        for f in range(amountOfFeatures):
            featureToGroup.append(g)

In [3]:
def decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity):
    model = Model("tree")
    model.parameters.randomseed = 42
    
    #datashape
    N = trainData.shape[0] # amount of samples
    I = range(1, N + 1) # all samples
    groupsN = trainData.shape[1] # amount of groupes
    groupsOfFeatures = trainData.nunique() # amount of features per group
    G = range(1, groupsN + 1) #groups
    J = range(1, len(trainDataDummies.columns) + 1) #features
    Iplus = np.where(samplePos == 1)[0]
    Iplus = [x+1 for x in Iplus]
    Imin = np.where(samplePos == 0)[0]
    Imin = [x+1 for x in Imin]
    
    
    #tree topology
    if (typeOfTree == 1):
        #simple tree of depth 2
        dNodesN = 3
        lNodesN = 4
        Bplus = [1, 3]
        Bmin = [2, 4]
        route = np.array([[1, 1, 0], [1, -1, 0], [-1, 0, 1], [-1, 0, -1]])
        nonAdjLeaf = [1]
        nonAdjLeafSymm = [1]
        AdjLeaf = [2, 3]
    if(typeOfTree == 2):
        #binary tree with depth 2,5
        dNodesN = 5
        lNodesN = 6
        Bplus = [1, 3, 5]
        Bmin = [2, 4, 6]
        route = np.array([[1, 1, 1, 0, 0], [1, 1, -1, 0, 0], [1, -1, 0, 1, 0], \
                          [1, -1, 0, -1, 0], [-1, 0, 0, 0, 1],  [-1, 0, 0, 0, -1]])
        nonAdjLeaf = [1, 2]
        nonAdjLeafSymm = [2]
        AdjLeaf = [3, 4, 5]
    if(typeOfTree == 3):
        #binary tree with depth 3
        dNodesN = 7
        lNodesN = 8
        Bplus = [1, 3, 5, 7]
        Bmin = [2, 4, 6, 8]
        route = np.array([[1, 1, 1, 0, 0, 0, 0], [1, 1, -1, 0, 0, 0, 0], [1, -1, 0, 1, 0, 0, 0], \
                         [1, -1, 0, -1, 0, 0, 0], [-1, 0, 0, 0, 1, 1, 0], [-1, 0, 0, 0, 1, -1, 0], \
                         [-1, 0, 0, 0, -1, 0, 1], [-1, 0, 0, 0, -1, 0, -1]])
        nonAdjLeaf = [1, 2, 5]
        nonAdjLeafSymm = [1, 2, 5]
        AdjLeaf = [3, 4, 6, 7]
    if(typeOfTree == 4):
        #binary tree with depth 3,5
        dNodesN = 7
        lNodesN = 8
        Bplus = [1, 3, 5, 7]
        Bmin = [2, 4, 6, 8]
        route = np.array([[1, 1, 1, 1, 0, 0, 0], [1, 1, 1, -1, 0, 0, 0], [1, 1, -1, 0, 1, 0, 0], \
                         [1, 1, -1, 0, -1, 0, 0], [1, -1, 0, 0, 0, 1, 0], [1, -1, 0, 0, 0, -1, 0], \
                         [-1, 0, 0, 0, 0, 0, 1], [-1, 0, 0, 0, 0, 0, -1]])
        nonAdjLeaf = [1, 2, 3]
        nonAdjLeafSymm = [3]
        AdjLeaf = [4, 5, 6, 7]
        
    #make nodes
    K = range(1, dNodesN + 1) #decision nodes
    B = range(1, lNodesN + 1) #leaf nodes

    # --- decision variables --- 
    idv = [(g, k) for g in G for k in K]
    if(relax): 
        v = model.continuous_var_dict(keys=idv, lb=0, ub=1, name="V") #relaxation of variables of paragraph 4.3.4
    else:
        v = model.binary_var_dict(keys=idv, name="V")
    idz = [(j,k) for j in J for k in K]

    if(relax):
        idzAdjLeaf = [(j,k) for j in J for k in AdjLeaf]
        idzNonAdjLeaf = [(j,k) for j in J for k in nonAdjLeaf]
        z = model.continuous_var_dict(keys=idzAdjLeaf, lb=0, ub=1, name="Z")
        zNonAdjLeaf = model.binary_var_dict(keys=idzNonAdjLeaf, name="Z")
        z.update(zNonAdjLeaf)
    else:
        idz = [(j,k) for j in J for k in K]
        z = model.binary_var_dict(keys=idz, name="Z")
    idc = [(b,i) for b in B for i in I]
    if(relax):
        c = model.continuous_var_dict(keys=idc,ub=1, name="C")
    else:
        c = model.binary_var_dict(keys=idc, name="C")
    idL = [(i,k) for i in I for k in K]
    L = model.binary_var_dict(keys=idL, name="L")
    idR = [(i,k) for i in I for k in K]
    R = model.binary_var_dict(keys=idL, name="R")
    if(numerical):
        idw = [(g,k) for k in K for g in numericalGroup]
        w = model.binary_var_dict(keys=idw, name="W")
    if(combCon):
        if(numerical):
            idw = [(g,k) for k in K for g in G if ((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard))]
            wcomb = model.binary_var_dict(keys=idw, name="W")
            w.update(wcomb)
        else:
            idw = [(g,k) for k in K for g in G if ((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard))]
            w = model.binary_var_dict(keys=idw, name="W")

    # --- constraints ---
    for k in K:
        model.add_constraint(model.sum(v[g, k] for g in G) == 1) #constraint (1)

    for k in K:
        for j in J:
            model.add_constraint(z[j,k] <= v[featureToGroup[j-1], k]) #constraint (2)

    for k in K:
        for i in I:
            model.add_constraint(L[i,k] == sum(z[j,k] for j in J if trainDataDummies.iloc[i-1,j-1] == 1)) #constraint (3)
            model.add_constraint(R[i,k] == 1 - L[i,k]) #constraint (4)
    
    if(strength):
        for i in I:
            for k in K:
                model.add_constraint(sum(c[b,i] for b in B if route[b-1, k-1] == 1) <= L[i,k]) #constraint (11)
                model.add_constraint(sum(c[b,i] for b in B if route[b-1, k-1] == -1) <= R[i,k]) #constraint (12)
    else:
        for b in B:
            for i in I:
                for k in K:
                    switch = route[b-1, k-1]
                    if switch == 1:
                        model.add_constraint(c[b,i] <= L[i,k]) #constraint (5)
                    elif switch == -1:
                        model.add_constraint(c[b,i] <= R[i,k]) #constraint (6)
        for i in I:
            model.add_constraint(model.sum(c[b,i] for b in B) == 1) #constraint (7)
    
    if(anchor): 
        for g in G:
            for k in nonAdjLeafSymm: #only for nonleafs with symmatrical subtree
                model.add_constraint(z[featureToGroup.index(g) + 1, k] == v[g,k]) #constraint (13)
                        
    if(ExclSameBranchFollowing):
        for k in K:
            for g in G:
                featureList = [i+1 for i, x in enumerate(featureToGroup) if x == g]
                model.add_constraint(model.sum(z[j,k] for j in featureList) >= v[g,k])
                model.add_constraint(model.sum( z[j,k] for j in featureList) <= (len(featureList) - 1)*v[g,k]) 
            
    if(numerical):
        #numerical value constraint 4.4
        for k in K:
            for g in numericalGroup:
                featureList = [i+1 for i, x in enumerate(featureToGroup) if x == g]
                if (len(featureList) > 1):
                    for j in featureList:
                        if(j == featureList[0]):    
                            model.add_constraint(z[j,k] >= z[j+1,k] - w[g,k])
                        elif(j == featureList[-1]):
                            model.add_constraint(z[j,k] >= z[j-1,k] - (1 - w[g,k]))
                        else:
                            model.add_constraint(z[j,k] >= z[j+1,k] - w[g,k])
                            model.add_constraint(z[j,k] >= z[j-1,k] - (1 - w[g,k]))                     
                
    if(combCon):
        for k in K:
            for g in G:
                if((g not in numericalGroup) and (groupsOfFeatures.iloc[g-1] > maxCard)):
                    featureList = [i for i, x in enumerate(featureToGroup) if x == g]
                    model.add_constraint(model.sum(z[j+1,k] for j in featureList) <= maxCard + (groupsOfFeatures.iloc[g-1] - maxCard)*(1-w[g,k]))
                    model.add_constraint(model.sum(z[j+1,k] for j in featureList) >= (groupsOfFeatures.iloc[g-1] - maxCard) - (groupsOfFeatures.iloc[g-1] - maxCard)*w[g,k])
        
        
    
    # --- objective ---
    model.objectiveTruePos = model.sum(model.sum(c[b,i] for b in Bplus) for i in Iplus)
    model.objectiveTrueNeg = model.sum(model.sum(c[b,i] for b in Bmin) for i in Imin)
    
    if(sensOn):
        
        model.add_constraint(model.sum(model.sum(c[b,i] for b in Bplus) for i in Iplus) >= math.ceil((1-sensitivity)*len(Iplus) ))
        
        totalObjective = model.objectiveTrueNeg
        
        sensitivity
    elif(specOn):
        model.add_constraint(model.sum(model.sum(c[b,i] for b in Bmin) for i in Imin) >= math.ceil((1-specificity)*len(Imin) ))

        totalObjective = model.objectiveTruePos
    else:
        totalObjective = model.objectiveTruePos + C*model.objectiveTrueNeg 
    
    model.maximize(totalObjective)
    
    return model

In [4]:
def treeValidation(solution, testDataDummies, trainDataDummies, typeOfTree):
    global LikArray
    global dfTest
    global decNodeFeatureDoubleList
    error = False
    decNodeFeatureDoubleList = []
    decNodeFeatureList = []
    
    if(typeOfTree == 1):
        nodeAmount = 3
    elif(typeOfTree == 2):
        nodeAmount = 5
    elif(typeOfTree == 3):
        nodeAmount = 7
    elif(typeOfTree == 4):
        nodeAmount = 7
    for currentNode in range(1, nodeAmount + 1):
        for feature in range(1, len(trainDataDummies.columns) + 1):
                value = sol.get_value('Z_' + str(feature) + '_' + str(currentNode))
                if(value > 0):
                    decNodeFeatureList.append(feature)
        decNodeFeatureDoubleList.append(decNodeFeatureList)
        decNodeFeatureList = []
    

    dfTest = pd.DataFrame(columns=trainDataDummies.columns)

    for col in dfTest.columns:
        if col in testDataDummies.columns: # copy columns from testdata to dataframe
            dfTest[col] = testDataDummies[col]
    dfTest = dfTest.fillna(0) #works
    
    #first part done
    
    #now find good route
    LikArray = np.zeros((len(dfTest.index), len(decNodeFeatureDoubleList)))
    
    for i in range(len(dfTest)):
        for k in range(len(decNodeFeatureDoubleList)):
            #check if Lik equals 1 or 0 
                for feature in decNodeFeatureDoubleList[k]:
                    if(dfTest.iloc[i,feature-1] == 1):
                        LikArray[i,k] = 1
                        
    totalCorrectClassified = 0
    if(typeOfTree == 1):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] == 1):
                if(LikArray[i,2-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
            else:
                if(LikArray[i,3-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
                        
    elif(typeOfTree == 2):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] == 1):
                if(LikArray[i,2-1] == 1):
                    if(LikArray[i,3-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                            
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                            
                else:
                    if(LikArray[i,4-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                            
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                            
            else:
                if(LikArray[i,5-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                        
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
                        
        
        
    elif(typeOfTree == 3):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] != 0):
                if(LikArray[i,2-1] != 0):
                    if(LikArray[i,3-1] != 0):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                else:
                    if(LikArray[i,4-1] != 0):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
            else:
                if(LikArray[i,5-1] != 0):
                    if(LikArray[i,6-1] != 0):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
                else:
                    if(LikArray[i,7-1] != 0):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
    elif(typeOfTree == 4):
        for i in range(len(dfTest)):
            if(LikArray[i,1-1] == 1):
                if(LikArray[i,2-1] == 1):
                    if(LikArray[i,3-1] == 1):
                        if(LikArray[i,4-1] == 1):
                            if(samplePosTest.iloc[i] == 1):
                                totalCorrectClassified += 1
                        else:
                            if(samplePosTest.iloc[i] == 0):
                                totalCorrectClassified += 1
                    else:
                        if(LikArray[i,5-1] == 1):
                            if(samplePosTest.iloc[i] == 1):
                                totalCorrectClassified += 1
                        else:
                            if(samplePosTest.iloc[i] == 0):
                                totalCorrectClassified += 1
                else:
                    if(LikArray[i,6-1] == 1):
                        if(samplePosTest.iloc[i] == 1):
                            totalCorrectClassified += 1
                    else:
                        if(samplePosTest.iloc[i] == 0):
                            totalCorrectClassified += 1
            else:
                if(LikArray[i,7-1] == 1):
                    if(samplePosTest.iloc[i] == 1):
                        totalCorrectClassified += 1
                else:
                    if(samplePosTest.iloc[i] == 0):
                        totalCorrectClassified += 1
                    
                
    else:
        print('error treetype not found...')
        
    return totalCorrectClassified / samplePosTest.size
    

In [5]:
solTime = 60*3 #2 minutes
C = 1  #weight to objective function
#ExclSameBranchFollowing  constraint (9) and (10)
numerical = False
combCon = False
maxCard = 3
sensOn = False
specOn = False
sensitivity = 0.95
specificity = 0.95

strength = True #constraint (11) and (12)
anchor = True #constraint (13)
relax = True

samplesize = 600
datasets = ['adult', 'breast', 'heloc','student']
typeOfTrees = [1, 2, 3, 4]
ExclSameBranchFollowings = [True, False]


for typeOfTree in typeOfTrees:
    for dataset in datasets:
        for ExclSameBranchFollowing in ExclSameBranchFollowings:
            listSolTime = []
            listTrainAcc = []
            listTestAcc = []
            for i in range(1, 6):
                seed = i
                
                DataPreperation(dataset, samplesize, False, seed)
                
                model = decisionTree(typeOfTree, C, ExclSameBranchFollowing, strength, anchor, relax, numerical, combCon, maxCard, sensOn, specOn, sensitivity, specificity)
                model.set_time_limit(solTime)
                
                sol = model.solve()
                
                objValue = sol.get_objective_value()
                listSolTime.append(model.solve_details.time)
                listTrainAcc.append( objValue / len(samplePos))
                listTestAcc.append(treeValidation(sol, testDataDummies, trainDataDummies, typeOfTree))

            meanSolTime = sum(listSolTime) / len(listSolTime)
            meanTrainAcc = sum(listTrainAcc) / len(listTrainAcc)
            meanTestAcc = sum(listTestAcc) / len(listTestAcc)
            print(dataset + ' exclSameBranching: ' + str(ExclSameBranchFollowing) +'  '+ str(typeOfTree) +' results: '+ str(meanTrainAcc*100) + ' (' + str(meanTestAcc*100) + ')' + '/' + str(meanSolTime))


adult exclSameBranching: True  1 results: 83.26666666666667 (79.54441913439635)/30.737200000000303
adult exclSameBranching: False  1 results: 83.26666666666667 (79.61275626423689)/31.847000000000115
breast exclSameBranching: True  1 results: 96.76666666666665 (94.39024390243901)/11.587599999998929
breast exclSameBranching: False  1 results: 96.76666666666665 (94.87804878048782)/11.03760000000184
heloc exclSameBranching: True  1 results: 72.93333333333335 (68.7250228217872)/180.0313999999984
heloc exclSameBranching: False  1 results: 72.96666666666667 (68.29698752408969)/180.03099999999978
student exclSameBranching: True  1 results: 78.98591549295773 (65.99999999999999)/95.71579999999668
student exclSameBranching: False  1 results: 78.98591549295773 (66.5)/119.2564000000013
adult exclSameBranching: True  2 results: 83.23333333333333 (78.22323462414579)/180.05280000000204
adult exclSameBranching: False  2 results: 83.53333333333333 (79.4760820045558)/180.0408000000025
breast exclSameBran

In [6]:
print(sol)

solution for: tree
objective: 249
V_1_2=1.000
V_1_3=1.000
V_1_4=1.000
V_1_6=1.000
V_1_7=1.000
V_2_1=1.000
V_3_5=1.000
Z_1_4=1.000
Z_1_6=1.000
Z_1_7=1.000
Z_2_4=1.000
Z_2_6=1.000
Z_2_7=1.000
Z_5_5=1.000
Z_6_5=1.000
Z_7_5=1.000
Z_8_5=1.000
Z_9_5=1.000
Z_1_2=1
Z_1_3=1
Z_2_2=1
Z_2_3=1
Z_3_1=1
Z_4_1=1
C_1_2=1.000
C_1_3=1.000
C_1_4=1.000
C_1_5=1.000
C_1_6=1.000
C_1_7=1.000
C_1_9=1.000
C_1_11=1.000
C_1_13=1.000
C_1_14=1.000
C_1_15=1.000
C_1_16=1.000
C_1_17=1.000
C_1_18=1.000
C_1_20=1.000
C_1_21=1.000
C_1_23=1.000
C_1_25=1.000
C_1_27=1.000
C_1_28=1.000
C_1_31=1.000
C_1_32=1.000
C_1_33=1.000
C_1_34=1.000
C_1_35=1.000
C_1_36=1.000
C_1_37=1.000
C_1_38=1.000
C_1_42=1.000
C_1_43=1.000
C_1_44=1.000
C_1_45=1.000
C_1_46=1.000
C_1_50=1.000
C_1_51=1.000
C_1_52=1.000
C_1_53=1.000
C_1_55=1.000
C_1_56=1.000
C_1_59=1.000
C_1_61=1.000
C_1_62=1.000
C_1_64=1.000
C_1_65=1.000
C_1_67=1.000
C_1_68=1.000
C_1_69=1.000
C_1_70=1.000
C_1_71=1.000
C_1_72=1.000
C_1_73=1.000
C_1_75=1.000
C_1_76=1.000
C_1_78=1.000
C_1_79=